In [1]:
%matplotlib notebook
import os, sys, time, datetime, collections
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import colors as mcolors
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import axes3d
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from timeit import default_timer as timer
from tqdm import tnrange, tqdm_notebook as tqdm

from coinmarketcap import Market
import ccxt
import options
config = options.Options()

In [2]:
from AI import Q_Trader
model = Q_Trader(config)

In [3]:
from slack import Slacker
slacker = Slacker(config)

In [4]:
if False:
    model.set_state_dim(8, 9)

    model.preRun()

    x = model.Q_value.eval(feed_dict={
        model.state_input1: np.array([[1,2,3,4,5,8,1,2]]),
        model.state_input2: [[1,2,3,4,5,6,7,8,9]]
    })
    print(x, np.argmax(x))

In [5]:
exchange = ccxt.cryptopia()
exchange.apiKey = config.cryptopia_api
exchange.secret = config.cryptopia_secret

In [6]:
# LOCAL CONFIG
coin = 'ETN'
pair = 'ETN/BTC'
verbose = False
# get first 10 mins of data. we are doing this and not ohlcv because we what 10 sec candles.
# these are important global variables...
update_secs = 10
max_updates = 60  # 10 mins
space = ' ' * 28
figure = None

In [7]:
def get_data(coin, pair):
    bal            = exchange.fetch_balance()['total'][coin]
    #time.sleep(1)
    # trade_history  = exchange.fetchMyTrades(pair)
    trade_history = None
    # time.sleep(1)
    # order_book     = exchange.fetch_order_book(pair)
    order_book = None
    time.sleep(1)
    curr_tick      = exchange.fetch_ticker(pair)
    time.sleep(1)
    btc_tick       = exchange.fetch_ticker('BTC/USDT')
    # time.sleep(1)
    # open_orders    = exchange.fetchOpenOrders(pair)
    open_orders = None
    return bal, trade_history, order_book, curr_tick, btc_tick, open_orders

In [8]:
def get_preroll():
    # be verbose... make sure pandas is succiencinked
    filename = 'preroll.csv'
    if os.path.exists(os.path.join(os.getcwd(), filename)):
        # try to load a good old version
        preroll = pd.read_csv(os.path.join(os.getcwd(), filename))
        print('Loaded classic file')
        return preroll

    preroll = pd.DataFrame(columns=['datetime','open','high',
                                'low','close','last', 
                                'volume','balance','btc_price'])
    try:
        # max_updates will be a self. and is global
        for i in tqdm(range(max_updates), desc='Prerolling:', leave=False):
            start_update_time = timer()
            bal, trade_history, order_book, curr_tick, btc_tick, open_orders = get_data(coin, pair)
            # need 9 elements
            datetime = curr_tick['datetime']
            open_ = curr_tick['open']
            high = curr_tick['high']
            low = curr_tick['low']
            close = curr_tick['close']
            last = curr_tick['last']
            vol = curr_tick['baseVolume']    
            btc_price = btc_tick['last']
            preroll = preroll.append([{ 'datetime': datetime,'open': open_,'high': high,
                                    'low': low,'close': close,'last': last,
                                    'volume': vol,'balance': bal,'btc_price': btc_price}])
            # get update runtime
            secs_to_update = timer() - start_update_time
            # update_secs will be a self. and is global
            secs_to_sleep = abs(update_secs - secs_to_update)
            secs_to_finish = 10 * (max_updates - i)
            time.sleep(secs_to_sleep)
    except:
        # if we get most of the preroll we can deal with it later.
        # this fails at like 98% sometimes... grr..
        pass
    preroll.to_csv(filename, index=False)
    print('saved a new file...')
    return preroll

In [9]:
def normalize(dataframe):
    # max updates will be self. and is global
    dataframe = dataframe.tail(max_updates)
    date_col = pd.Series(dataframe.pop('datetime'))
    
    def unzero(df):
        df = df.replace(0,'NaN')
        df = df.dropna(how='all',axis=0)
        df = df.replace('NaN', 0)
        df.len = len(df)
        return df
    
    one_min_change = unzero(dataframe.pct_change(periods=6))
    five_min_change = unzero(dataframe.pct_change(periods=6 * 5))
    return dataframe, one_min_change, five_min_change, date_col

In [10]:
def get_next_sample(old_dataset):
    sample = get_data(coin, pair)  # bal is 0, curr_tick is 3, btc_tick is 4
    new_dataset = old_dataset.append([{
        'datetime': sample[3]['datetime'],
        'open': sample[3]['open'],
        'high': sample[3]['high'],
        'low': sample[3]['low'],
        'close': sample[3]['close'],
        'last': sample[3]['last'],
        'volume': sample[3]['baseVolume'],
        'balance': sample[0],
        'btc_price': sample[4]['last']
                              }])
    return new_dataset

In [11]:
def setup_graphs(datasets):
    dates = datasets[3]
    sets_to_graph = datasets[0:2]
    # first just do a bar plot of real vol data
    # then a line for real last
    # this figure will have to be global self.
    figure = plt.figure()
    
    # first ax
    Fminset = datasets[2]
    Fmin_3d_axe = figure.add_subplot(111, projection='3d')
    Fmin_3d_axe.plot_surface(Fminset['close'],
                               dates[:-len(Fminset)],
                                Fminset['btc_price'],
                               rstride=10,
                               cstride=10)
    plt.show()

In [12]:
def update_graphs(datasets):
    '''UPDATE 3D GRAPH'''
    # create updateable object
    def update_lines(num, dataLines, lines):
        for line, data in zip(lines, dataLines):
            # NOTE: there is no .set_data() for 3 dim data...
            line.set_data(data[0:2, :num])
            line.set_3d_properties(data[2, :num])
        return lines
    # Creating the Animation object
    line_ani = animation.FuncAnimation(fig, update_lines, 25, 
                                       fargs=(data, lines),
                                       interval=5, blit=False)
    """UPDATE NEXT GRAPH"""
    pass

In [13]:
def make_input_data(datasets):
    sets = datasets[1:-1]
    inputs = []
    for i in sets:
        _set = i
        X1 = pd.Series(_set['open']).astype(float).as_matrix()
        X2 = pd.Series(_set['close']).astype(float).as_matrix()
        X3 = pd.Series(_set['last']).astype(float).as_matrix()
        X4 = pd.Series(_set['volume']).astype(float).as_matrix()
        X5 = pd.Series(_set['btc_price']).astype(float).as_matrix()
        X6 = np.append([X1, X2, X3, X4], X5)
        inputs.append(X6)
    return inputs

In [14]:
class StockEnv(object):
    def __init__(self, data):
        self.action_space = ['b','s','n']
        self.n_actions = len(self.action_space)
        self.data1 = data[0]
        self.data2 = data[1]
        self.step = 0
        self.max_len = min(len(self.data1), len(self.data2))

    def reset(self):
        data = self.data1[self.step], self.data2[self.step]
        return data

    def gostep(self, action):
        self.step +=1
        first_data = self.data[0], self.data2[0]
        last_data = self.data[self.step-1], self.data2[self.step-1]
        next_data = self.data[self.step], self.data2[self.step]
        reward = 0

        if action == 0: # buy
            pass
        elif action == 1: #sell
            pass
        else:
            reward = 1
            # pass  # nothing

        if self.step == len(self.max_len):
            done = True
        else:
            done = False
        
        return next_data, reward, done

In [15]:
def tensorflowit(input_data):
    env = StockEnv(input_data)
    data_step = env.reset()
    while True:
        action = model.egreedy_action(data_step)
        next_data_step, reward, done = env.gostep(action)
        model.train(data_step, action, reward, next_data_step, done)
        data_step = next_data_step
        if done:
            break      

In [16]:
def get_reward(action, state):
    # btc price trend
    btc = []
    coin = []
    vol = []
    for i in range(len(state[0])):
        if i % 5 == 0:
            coin.append('{:.4f}'.format(float(state[0][i-2])))
            vol.append('{:.4f}'.format(float(state[0][i-1])))
            btc.append('{:.4f}'.format(float(state[0][i])))
    btctrend = np.array(btc, dtype=np.float64).sum()
    voltrend = np.array(vol, dtype=np.float64).sum()
    cointrend = np.array(coin, dtype=np.float64).sum()
    rightness = np.array([btctrend, cointrend, voltrend], dtype=np.float64).sum()
    # print('btc trend: {:.3f}'.format(btctrend))
    # print('coin trend: {:.3f}'.format(cointrend))
    # print('vol trend: {:.3f}'.format(voltrend))
    # print('Agg Trend: {:.3f}'.format(rightness))
    right_answer = 2  # do nothing
    if rightness > 0.02:
        right_answer = 0  # buy if going up
    elif rightness < -0.02:
        right_answer = 1  # sell if going down
        
    if action == right_answer:
        reward = 1
    else: reward = -1
    print('action {}, right {}, reward {}'.format(action, right_answer, reward), end='\r')
    return reward

In [17]:
preroll = last_set = get_preroll()

Loaded classic file


In [18]:
datasets = normalize(preroll)
input_data = make_input_data(datasets)
model.set_state_dim(input_data[0].shape[0], input_data[1].shape[0])
model.preRun()

In [19]:
if False:
    action = model.egreedy_action(input_data)

    reward = get_reward(action, input_data)

    last_input_data = input_data
    data = get_next_sample(preroll)

    model.train(last_input_data, action, reward, next_input_data)

In [20]:
loop_num = 0
kill_max_loops = 3 # 
running = True
last_set = preroll
next_set = None
last_input = None
next_input = None
while running:
    print('Top of loop #{}. I want this to take less than 10 secs..{}'.format(loop_num, space), end='\r')
    start_loop_time = timer()
    next_set = get_next_sample(last_set)
    print('Update took {:.2f} secs{}'.format(timer()-start_loop_time, space), end='\r')
    datasets = normalize(next_set)  # [dataset(60), one_min_change(54), five_min_change(30), date_col(60)]
    print('Normalize data took {:.2f} secs{}'.format(timer()-start_loop_time, space), end='\r')
    last_input = next_input
    next_input = make_input_data(datasets)
    print('Stacking data took {:.2f} secs{}'.format(timer()-start_loop_time, space), end='\r')
    if loop_num <= 0:
        print('getting 2 instances to start with', end='\r')
        loop_num += 1
        continue
    action = model.egreedy_action(last_input)
    reward = get_reward(action, last_input)
    
    start_training_time = timer()
    model.train(last_input, action, reward, next_input)
    training_time = timer() - start_training_time
    print('Training time: {:2f} secs{}'.format(training_time, space), end='\r')
        
    #if loop_num >= kill_max_loops:  # if run out of time
    #    running = False
        
    loop_time = timer()-start_loop_time
    sleep_time = update_secs - loop_time
    if sleep_time > 0:
        time.sleep(sleep_time/2)
        print("loopTime: {:.2f}, Sleep: {:.2f}{}".format(loop_time, sleep_time, space), end='\r')
        time.sleep(sleep_time/2)
    else:
        print("loopTime: {:.2f} going too slow.{}".format(timer()-start_loop_time, space), end='\r')
        time.sleep(.25)
    loop_num += 1
    
# game over
print('AlphaGriffin | 2018{}'.format(space))
    

RequestTimeout: cryptopia GET https://www.cryptopia.co.nz/api/GetMarket/5662 request timeout